In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
g4_df = pd.read_csv("../muon_rtd_2023-06-01_165701/g4_output.txt")
resets_df = pd.read_csv("../muon_rtd_2023-06-01_165701/resets_output.txt")

In [ ]:
xi_muon = []
yi_muon = [] 
zi_muon = []
E_muon = []

for i in range(100):
    xi_muon.append(g4_df[g4_df.event == i].xi.iloc[0])
    yi_muon.append(g4_df[g4_df.event == i].yi.iloc[0])
    zi_muon.append(g4_df[g4_df.event == i].zi.iloc[0])  
    E_muon.append(g4_df[g4_df.event == i].E.iloc[0])  
    
int_stats_df = pd.DataFrame()
int_stats_df['xi_muon']=pd.Series(xi_muon)
int_stats_df['yi_muon']=pd.Series(yi_muon)
int_stats_df['zi_muon']=pd.Series(zi_muon)
int_stats_df['E_muon']=pd.Series(E_muon)
int_stats_df.to_pickle('int_stats_df.pkl')

In [ ]:
event0_g4_df = g4_df[['event','xi','xf','yi','yf','zi','zf','ti','tf','E','ParticleID']][g4_df.event == 0].copy()
event0_g4_df.to_pickle('event0_g4_df.pkl')

In [ ]:
resets_df_micro = resets_df.copy()
resets_df_micro['reset_time'] = resets_df_micro.reset_time.mul(1e+6)
resets_df_micro.to_pickle('resets_df_micro.pkl')

In [ ]:
event0_resets_df_micro = resets_df_micro[['event','pixel_x','pixel_y','reset_time','TSLR','nMCParticles']][resets_df_micro.event == 0].copy()
event0_resets_df_micro.to_pickle('event0_resets_df_micro.pkl')

In [ ]:
def rms(array):
    return np.sqrt(np.sum((array - np.mean(array))**2) / len(array))

pix_x_event = []
pix_y_event = []
active_pixels_event = []

mean_resets = []
rms_resets = []
active_pix_x = []
active_pix_y = []
event_num = []
resets_num = []

for k in range(0,100):
    for i in range(len(resets_df_micro[resets_df_micro.event == k].reset_time)):
        if (resets_df_micro[resets_df_micro.event == k].pixel_x.iloc[i] in pix_x_event) and (resets_df_micro[resets_df_micro.event == k].pixel_y.iloc[i] in pix_y_event):
            pass
        else:
            pix_x_event.append(resets_df_micro[resets_df_micro.event == k].pixel_x.iloc[i])
            pix_y_event.append(resets_df_micro[resets_df_micro.event == k].pixel_y.iloc[i])
    
    active_pixels_event.append(len(pix_x_event))
    for m in range(len(pix_x_event)):
        reset_subset = resets_df_micro[(resets_df_micro.event == k) & (resets_df_micro.pixel_x == pix_x_event[m]) & (resets_df_micro.pixel_y == pix_y_event[m])].reset_time.to_numpy()
        if len(reset_subset) > 0:
            mean_value = np.mean(reset_subset)  # Calculate the mean of the current subset
            rms_value = rms(reset_subset)  # Calculate the RMS of the current subset
            mean_resets.append(mean_value)
            rms_resets.append(rms_value)
            resets_num.append(len(reset_subset))
            event_num.append(k)
            active_pix_x.append(pix_x_event[m])
            active_pix_y.append(pix_y_event[m])      
            
    pix_x_event = []
    pix_y_event = []
    reset_subset = []

    
active_pixels_df = pd.DataFrame()   
active_pixels_df['active_pixels']=pd.Series(active_pixels_event)
active_pixels_df.to_pickle('active_pixels_df.pkl')

pixel_stats_df = pd.DataFrame()
pixel_stats_df['event']=pd.Series(event_num)
pixel_stats_df['pix_x']=pd.Series(active_pix_x)
pixel_stats_df['pix_y']=pd.Series(active_pix_y)
pixel_stats_df['resets']=pd.Series(resets_num)
pixel_stats_df['mean_ToA']=pd.Series(mean_resets)
pixel_stats_df['RMS']=pd.Series(rms_resets)
pixel_stats_df.to_pickle('pixel_stats_df.pkl')

In [ ]:
print(pixel_stats_df)

In [ ]:
print(active_pixels_df)

In [ ]:
print(pixel_stats_df[pixel_stats_df.event == 0])